In [1]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [2]:
!kaggle datasets download -d abhinavmoudgil95/short-jokes

short-jokes.zip: Skipping, found more recently modified local copy (use --force to force download)


In [3]:
import zipfile
zip_ref = zipfile.ZipFile('/content/short-jokes.zip', 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [4]:
import pandas as pd
jokes = pd.read_csv('shortjokes.csv')
jokes.shape

(231657, 2)

In [5]:
jokes = jokes['Joke'].sample(6000).values
jokes

array(['How does a psychic cokehead tell the future? With a magic 8-ball',
       'What do you call a programming language designed for women? An object oriented programming language.',
       "When does a rabbit go exactly as fast as a train? When it's on the train.",
       ...,
       'What do you call it when a golfer misses the hole? Fore-play',
       'What is the most faithful insect ? A flea once they find someone they like they stick to them !',
       "What does a terrorist tell hes son? Aim to the sky, maybe you'll shoot a plane."],
      dtype=object)

In [6]:
len(jokes)

6000

In [7]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer

#Tokenize the text
tokenizer=Tokenizer()
tokenizer.fit_on_texts(jokes)


In [8]:
#Show mapping of workds with numbers
print(tokenizer.word_index)
print(len(tokenizer.word_index))

{'a': 1, 'the': 2, 'i': 3, 'to': 4, 'you': 5, 'and': 6, 'in': 7, 'my': 8, 'what': 9, 'of': 10, 'is': 11, 'it': 12, 'do': 13, 'me': 14, 'on': 15, 'was': 16, 'for': 17, 'have': 18, 'with': 19, 'that': 20, 'why': 21, 'like': 22, 'he': 23, 'when': 24, 'your': 25, 'are': 26, 'how': 27, 'did': 28, 'they': 29, 'so': 30, 'if': 31, 'just': 32, 'an': 33, "i'm": 34, 'but': 35, 'because': 36, 'at': 37, 'be': 38, 'call': 39, 'get': 40, "it's": 41, 'out': 42, 'his': 43, 'this': 44, 'about': 45, "don't": 46, 'one': 47, 'can': 48, 'not': 49, 'her': 50, 'up': 51, 'all': 52, 'she': 53, 'people': 54, 'who': 55, 'say': 56, 'no': 57, 'does': 58, "what's": 59, 'know': 60, 'into': 61, 'man': 62, 'them': 63, 'we': 64, 'there': 65, 'said': 66, 'from': 67, 'as': 68, 'had': 69, 'has': 70, "can't": 71, 'time': 72, 'by': 73, 'their': 74, 'got': 75, 'two': 76, 'make': 77, 'joke': 78, 'between': 79, 'go': 80, 'him': 81, 'other': 82, 'only': 83, 'never': 84, 'says': 85, 'want': 86, 'take': 87, 'think': 88, 'would': 8

In [9]:
total_words = len(tokenizer.word_index)+1
total_words

11554

In [10]:
#Create input sequence and labels
input_seq=[]
i=0
for joke in jokes:
  token_list = tokenizer.texts_to_sequences([joke])[0]
  for i in range(1,len(token_list)):
    ngram=token_list[:i+1]
    input_seq.append(ngram)

In [11]:
max_sequence_length = max([len(i) for i in input_seq])
max_sequence_length

46

In [12]:
#padding input sequence
from tensorflow.keras.preprocessing.sequence import pad_sequences

padded_input_seq=pad_sequences(input_seq,maxlen=max_sequence_length,padding='pre')

In [13]:
padded_input_seq[:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          27,   58],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,   27,
          58,    1],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,   27,   58,
           1, 1325],
       [   0,    0,    0,    0,    0,    0,    0,

In [14]:
X=padded_input_seq[:,:-1]
y=padded_input_seq[:,-1]

In [15]:
X.shape

(99722, 45)

In [16]:
y.shape

(99722,)

In [17]:
from tensorflow.keras.utils import to_categorical
y = to_categorical(y,num_classes=total_words)

In [18]:
y.shape

(99722, 11554)

In [19]:
from keras import Sequential
from keras.layers import Dense,GRU,Embedding

model=Sequential()
model.add(Embedding(total_words,100,input_length=max_sequence_length-1))
model.add(GRU(150))
model.add(Dense(total_words,activation='softmax'))


In [20]:
# Compile the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
  # Train the model
model.fit(X, y, epochs=30, verbose=1)

Epoch 1/30
3117/3117 [==============================] - 64s 19ms/step - loss: 6.8203 - accuracy: 0.0761
Epoch 2/30
3117/3117 [==============================] - 37s 12ms/step - loss: 5.9107 - accuracy: 0.1329
Epoch 3/30
3117/3117 [==============================] - 36s 12ms/step - loss: 5.3040 - accuracy: 0.1658
Epoch 4/30
3117/3117 [==============================] - 36s 11ms/step - loss: 4.7549 - accuracy: 0.1960
Epoch 5/30
3117/3117 [==============================] - 37s 12ms/step - loss: 4.2377 - accuracy: 0.2370
Epoch 6/30
3117/3117 [==============================] - 36s 12ms/step - loss: 3.7584 - accuracy: 0.2891
Epoch 7/30
3117/3117 [==============================] - 35s 11ms/step - loss: 3.3289 - accuracy: 0.3487
Epoch 8/30
3117/3117 [==============================] - 36s 12ms/step - loss: 2.9559 - accuracy: 0.4074
Epoch 9/30
3117/3117 [==============================] - 36s 11ms/step - loss: 2.6422 - accuracy: 0.4593
Epoch 10/30
3117/3117 [==============================] - 35s 11m

In [36]:
# Generate a new joke based on a seed text
import numpy as np
def generate_joke(seed_text, next_words, model, max_sequence_length):
    for _ in range(next_words):
        token_list = tokenizer.texts_to_sequences([seed_text])[0]
        token_list = pad_sequences([token_list], maxlen=max_sequence_length-1, padding='pre')
        predicted = np.argmax(model.predict(token_list), axis=-1)
        output_word = ""
        for word, index in tokenizer.word_index.items():
            if index == predicted:
                output_word = word
                break
        seed_text += " " + output_word
    return seed_text

# Seed text to start the joke
seed_text = "Can you write a joke for me as I am not very good at it"

# Number of words to generate in the joke
next_words = 15

# Generate and print the new joke
generated_joke = generate_joke(seed_text, next_words, model, max_sequence_length)
print("Generated Joke:", generated_joke)

1/1 [==============================] - 0s 31ms/step
Generated Joke: Can you write a joke for me as I am not very good at it like anyone unless of people with this birthday is going to my facebook i'm holding
